In [1]:
# %load honest_net.py
import numpy as np
import pandas as pd

df = pd.read_csv("../../data/honest_net.tsv", sep="\t")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]


# C&P from https://stackoverflow.com/a/39513799
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad / np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


def parse_array(s):
    try:
        return np.fromstring(s, dtype=float, sep="|")
    except TypeError:
        return np.array([float("nan")])


def expand(row):
    compute = parse_array(row.compute)
    weakest = np.argmin(compute)
    strongest = np.argmax(compute)
    d = {}

    def wsg(k, v):
        d[k + "_weakest"] = v[weakest]
        d[k + "_strongest"] = v[strongest]
        d[k + "_gini"] = gini(v)

    rcompute = compute / np.sum(compute)
    wsg("compute", rcompute)
    activations = parse_array(row.activations)
    assert np.sum(activations) == row.number_activations or row.error
    ractivations = activations / np.sum(activations)
    wsg("activations", ractivations)
    reward = parse_array(row.reward)
    rreward = reward / np.sum(reward)
    wsg("reward", rreward)
    efficiency = rreward / ractivations
    wsg("efficiency", efficiency)
    d["activations_compute_gini_delta"] = d["activations_gini"] - d["compute_gini"]
    d["reward_activations_gini_delta"] = d["reward_gini"] - d["activations_gini"]
    return d


df = df.join(df.apply(expand, axis=1, result_type="expand"))

## Incentive Schemes for Georges  Protocol

In [2]:
d = df[df.protocol == "george"]

### Efficiency of the weakest miner

We pick the weakest miner by compute power and caluculate its

efficiency = relative reward / relative activations

In [3]:
eff = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="efficiency_weakest",
)
eff.to_latex(buf="../../tex/george-reward-inequality/tab/efficiency-weakest.tex")
eff

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.992089  0.992089  0.992089  0.992089  0.992089
               2    0.979370  0.984524  0.984524  0.984524  0.984524
               4    0.969144  0.982597  0.979192  0.968015  0.968317
               8    0.950177  0.985601  0.979427  0.943237  0.945423
               16   0.881597  0.985766  0.980688  0.880016  0.881139
               32   0.710455  0.978602  0.973072  0.755802  0.756989
               64   0.706918  0.976288  0.972495  0.564144  0.566400
               128  0.331374  0.977925  0.974713  0.295636  0.295836
60.0           1    0.997408  0.997408  0.997408  0.997408  0.997408
               2    0.984510  0.992090  0.992090  0.992090  0.992090
               4    0.982581  0.991625  0.989028  0.984381  0.985651
               8    0.973575  0.990599  0.987998  0.969322  0.969482
               16   0.928676  0.990979  0.987295  0.942620  0.944108
               32   0.920825  0.991242  0.989394  0.877469  0.876672
               64   0.766019  0.988678  0.986878  0.758654  0.758870
               128  0.579689  0.988544  0.986023  0.558926  0.560450
120.0          1    0.998914  0.998914  0.998914  0.998914  0.998914
               2    1.014695  0.995182  0.995182  0.995182  0.995182
               4    1.004402  0.996496  0.994990  0.991689  0.992175
               8    1.013124  0.996954  0.996247  0.988202  0.987836
               16   0.948885  0.995421  0.993398  0.969764  0.970362
               32   0.955805  0.996540  0.995013  0.942970  0.943270
               64   0.931711  0.994514  0.993190  0.874792  0.875064
               128  0.670985  0.995825  0.995141  0.760368  0.760185
300.0          1    0.999150  0.999150  0.999150  0.999150  0.999150
               2    1.001489  0.998209  0.998209  0.998209  0.998209
               4    0.996913  0.998882  0.998347  0.997533  0.997795
               8    0.999812  0.998644  0.998050  0.994555  0.994648
               16   1.004682  0.996407  0.995822  0.983355  0.983204
               32   0.981335  0.998210  0.997855  0.972421  0.971988
               64   0.894643  0.997918  0.997932  0.949737  0.949149
               128  0.974220  0.998266  0.997847  0.906608  0.906557
600.0          1    0.999739  0.999739  0.999739  0.999739  0.999739
               2    1.009494  0.999407  0.999407  0.999407  0.999407
               4    1.004113  0.999346  0.999459  0.998562  0.998153
               8    1.011927  0.999362  0.998755  0.996917  0.997235
               16   0.989324  0.999081  0.998745  0.992346  0.992286
               32   1.008482  0.999248  0.999094  0.987511  0.987288
               64   1.074113  0.998533  0.997959  0.970999  0.971177
               128  1.090268  0.998422  0.998075  0.948344  0.948419

### Reward Gini Fairness

We compare the rewards of all miners using the Gini coefficient. A lower coefficient implies more fairness. All numbers should be compared to the Gini coefficient of the compute powers.

In [4]:
d.compute_gini.unique()

array([0.3])

In [5]:
gini = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="reward_gini",
)
gini.to_latex(buf="../../tex/george-reward-inequality/tab/reward-gini.tex")
gini

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.301341  0.301341  0.301341  0.301341  0.301341
               2    0.303348  0.302833  0.302833  0.302833  0.302833
               4    0.305474  0.303605  0.304212  0.305758  0.305595
               8    0.311103  0.303932  0.304862  0.311636  0.311487
               16   0.323884  0.303864  0.304903  0.323450  0.323198
               32   0.346819  0.305135  0.306265  0.350495  0.350003
               64   0.372177  0.305460  0.306365  0.397904  0.397305
               128  0.442164  0.307089  0.308070  0.478060  0.476919
60.0           1    0.300484  0.300484  0.300484  0.300484  0.300484
               2    0.301437  0.301341  0.301341  0.301341  0.301341
               4    0.302621  0.302038  0.302462  0.303290  0.303116
               8    0.306390  0.302301  0.302810  0.306307  0.306242
               16   0.312242  0.302239  0.302846  0.312219  0.312138
               32   0.322248  0.301946  0.302523  0.324528  0.324405
               64   0.342347  0.302368  0.302847  0.349747  0.349593
               128  0.376654  0.302672  0.303197  0.398890  0.398494
120.0          1    0.300198  0.300198  0.300198  0.300198  0.300198
               2    0.301106  0.301277  0.301277  0.301277  0.301277
               4    0.301921  0.301307  0.301461  0.301842  0.301811
               8    0.303194  0.301422  0.301693  0.303501  0.303474
               16   0.308036  0.301487  0.301786  0.306340  0.306309
               32   0.307197  0.300628  0.300970  0.311258  0.311190
               64   0.318305  0.301116  0.301413  0.324766  0.324706
               128  0.347114  0.300804  0.301052  0.349468  0.349395
300.0          1    0.299957  0.299957  0.299957  0.299957  0.299957
               2    0.300812  0.300929  0.300929  0.300929  0.300929
               4    0.300774  0.300993  0.301082  0.301268  0.301239
               8    0.301849  0.300814  0.300894  0.301467  0.301466
               16   0.300827  0.300614  0.300761  0.302722  0.302694
               32   0.303393  0.300615  0.300725  0.304878  0.304892
               64   0.307905  0.300898  0.301011  0.310345  0.310355
               128  0.310791  0.300023  0.300185  0.318613  0.318539
600.0          1    0.299879  0.299879  0.299879  0.299879  0.299879
               2    0.300703  0.300752  0.300752  0.300752  0.300752
               4    0.300425  0.300764  0.300815  0.300915  0.300897
               8    0.299081  0.300844  0.300920  0.301272  0.301246
               16   0.303822  0.300435  0.300527  0.301518  0.301492
               32   0.305745  0.299891  0.299963  0.302127  0.302125
               64   0.298100  0.300418  0.300500  0.305015  0.304999
               128  0.304391  0.300559  0.300646  0.310075  0.310049